In [23]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Ensure the environment variables are set
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')
huggingface_api_key = os.getenv('HUGGINGFACE_API_KEY')

if not langchain_api_key:
    raise ValueError("LANGCHAIN_API_KEY is not set in the environment variables.")
if not huggingface_api_key:
    raise ValueError("HUGGINGFACE_API_KEY is not set in the environment variables.")

# Set environment variables for the application
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
os.environ['HUGGINGFACE_API_KEY'] = huggingface_api_key

In [24]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [25]:
# Load Documents (use PyPDFLoader for PDF)
file_path = r"/Users/saifmohammed/Desktop/CSE299-Junior-Design-Project/LLM-1/PDFs/Diabetes_Care_BADAS_guideline2019.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

docs[0].page_content[:1000]

'DIABETES CARE \nBADAS Guideline 2019 \n          \n  \n   \n  \n   P|) \nDAS GUELINE ON Man \nDELIT IGEMEN \n  \nA Joint Initiative of \nDiabetic Association of Bangladesh \nNCDC Program, Directorate General of Health Services'

In [26]:
# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

for i, chunk in enumerate(splits[:3]):  # Show the first 3 chunks
    print(f"\n--- Chunk {i+1} ---")
    print(chunk.page_content[:1000])  # Print the first 1000 characters of the chunk
    print("\n" + "-"*70 + "\n")  # Separator between chunks


--- Chunk 1 ---
DIABETES CARE 
BADAS Guideline 2019 
          
  
   
  
   P|) 
DAS GUELINE ON Man 
DELIT IGEMEN 
  
A Joint Initiative of 
Diabetic Association of Bangladesh 
NCDC Program, Directorate General of Health Services

----------------------------------------------------------------------


--- Chunk 2 ---
DIABETES CARE 
BADAS Guideline 2019 
  
A Joint Initiative of 
Diabetic Association of Bangladesh 
NCDC Program, Directorate General of Health Services 
  
Diabetes Care: BADAS Guideline 2019 HEI! 1

----------------------------------------------------------------------


--- Chunk 3 ---
DIABETES CARE: BADAS GUIDELINE 2019 
Convener: Prof A K Azad Khan 
Chairman: Prof Hajera Mahtab 
Members of the steering committee 
Prof Dr AHM Enayet Hossain 
Prof Akhtar Hussain 
Prof Zafar Anmed Latif 
Prof Tofail Ahmed 
Prof Laique Ahmed Khan 
Prof Nazrul Islam Siddiqui 
Prof Md Hafizur Rahman 
Prof Abdus Saleque Mollah 
Prof Md Farid Uddin 
Prof M A Jalil Ansary 
Prof Dr MA Samad 


In [27]:
from langchain_huggingface import HuggingFaceEmbeddings

# Use HuggingFace Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)
retriever = vectorstore.as_retriever()

In [29]:
from crewai import Agent, Task, Crew
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import OllamaLLM

In [30]:
from langchain.tools import Tool
from crewai import Agent

# Define a function that retrieves documents
def retrieve_docs(query):
    retrieved_docs = retriever.get_relevant_documents(query)
    return "\n\n".join([doc.page_content for doc in retrieved_docs])

# Wrap the function in a CrewAI-compatible Tool
retriever_tool = Tool(
    name="Retriever Tool",
    func=retrieve_docs,
    description="Fetches relevant document snippets for a given query."
)

# Create the Retriever Agent
retriever_agent = Agent(
    role="Retriever",
    goal="Retrieve the most relevant information from the knowledge base",
    backstory="You fetch high-quality document snippets for answering queries.",
    verbose=True,
    tools=[retriever_tool],  # Use the wrapped tool
)

In [31]:
from langchain_ollama import OllamaLLM

# Define LLM-based response generation tool
llm = OllamaLLM(model="llama3.2")

def generate_response(query):
    return llm.invoke(query)

llm_tool = Tool(
    name="LLM Generator Tool",
    func=generate_response,
    description="Generates a detailed response based on retrieved context."
)

# Create the Generator Agent
generator_agent = Agent(
    role="Generator",
    goal="Generate detailed, informative responses based on the retrieved knowledge.",
    backstory="You analyze the retrieved context and provide well-structured, insightful answers.",
    verbose=True,
    tools=[llm_tool],  # Use the wrapped LLM tool
)

In [32]:
orchestrator_agent = Agent(
    role="Orchestrator",
    goal="Coordinate the retriever and generator to answer queries efficiently.",
    backstory="You ensure smooth collaboration between retriever and generator agents for seamless question-answering.",
    verbose=True,
)

In [33]:
os.environ["OPENTELEMETRY_PYTHON_NO_LOGGING"] = "true"


from crewai import Crew

crew = Crew(
    agents=[retriever_agent, generator_agent, orchestrator_agent],
    verbose=True
)

In [34]:
from crewai import Task

rag_task = Task(
    description="Retrieve relevant information and generate a detailed response to the user's question.",
    agent=orchestrator_agent,  # The orchestrator agent will coordinate everything
    expected_output="A well-structured, informative answer based on retrieved knowledge."
)

In [35]:
# Initialize the Crew with agents and tasks
crew = Crew(
    agents=[retriever_agent, generator_agent, orchestrator_agent],
    tasks=[rag_task]
)

# Execute the RAG process
result = crew.kickoff()

# Print the answer
print("\n\n🔹 Answer:\n", result)

ERROR:root:LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable


# Agent: Orchestrator
## Task: Retrieve relevant information and generate a detailed response to the user's question.


LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

 Error during LLM call: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable


AuthenticationError: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable